In [1]:
!pip install omnixai pillow<10.0


/bin/bash: ligne 1: 10.0: Aucun fichier ou dossier de ce nom


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import matplotlib.pyplot as plt
import PIL
import gc

import urllib.request
import logging
import os

import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import cv2
from scipy.ndimage import filters
from torch import nn

import clip

from matplotlib.backends.backend_pdf import PdfPages
import torch.distributed as dist
from lavis.common.dist_utils import get_rank, get_world_size, is_main_process, is_dist_avail_and_initialized
from lavis.common.logger import MetricLogger, SmoothedValue
from lavis.common.registry import registry
from lavis.datasets.data_utils import prepare_sample
from lavis.models.clip_models.model import CLIP, load_state_dict, load_openai_model
from lavis.processors.clip_processors import ClipImageTrainProcessor
from lavis.processors.blip_processors import BlipCaptionProcessor
from transformers import CLIPProcessor, CLIPModel

from omnixai.data.text import Text
from omnixai.data.image import Image
from omnixai.data.multi_inputs import MultiInputs
from omnixai.preprocessing.image import Resize
from omnixai.explainers.vision_language.specific.gradcam import GradCAM

In [3]:
def load_Fmodel_clip(chekpoint=True, vit=True):
    preprocess=None
    if vit:
        model_cfg ={
            "embed_dim": 768,
            "quick_gelu": True,
            "vision_cfg": {
                "image_size": 336,
                "layers": 24,
                "width": 1024,
                "patch_size": 14
            },
            "text_cfg": {
                "context_length": 77,
                "vocab_size": 49408,
                "width": 768,
                "heads": 12,
                "layers": 12
            }
        }
        model = CLIP(**model_cfg, add_cls_token=False)
        if chekpoint:
            checkpoint_path = "lavis/checkpoint_best (1).pth"
            model.load_state_dict(load_state_dict(checkpoint_path))
        _, preprocess = clip.load("ViT-L/14")
    else:
        model, preprocess = clip.load("RN50", device="cuda")
    del _
    return model, preprocess

In [4]:
def forward(model,dico):
    im,txt = dico['im'],dico['txt']
    txt_enc = BlipCaptionProcessor(max_words=77)(txt)
    txt_enc= model.tokenizer(txt_enc, add_cls_token=True)
    txt_enc,txt_enc_class=model.encode_text(txt_enc.to(dtype=torch.int64))
    txt_enc=F.normalize(txt_enc, dim=-1)

    image=ClipImageTrainProcessor(image_size=336)(im)
    feat_im = model.encode_image(image[None])
    feat_im =F.normalize(feat_im, dim=-1)
    del im,txt, txt_enc_class, image
    return txt_enc @ feat_im.T



In [5]:
def normalize(x: np.ndarray) -> np.ndarray:
    # Normalize to [0, 1].
    x = x - x.min()
    if x.max() > 0:
        x = x / x.max()
    return x

# Modified from: https://github.com/salesforce/ALBEF/blob/main/visualization.ipynb
def getAttMap(img, attn_map, blur=True):
    if blur:
        attn_map = filters.gaussian_filter(attn_map, 0.02*max(img.shape[:2]))
    attn_map = normalize(attn_map)
    cmap = plt.get_cmap('jet')
    attn_map_c = np.delete(cmap(attn_map), 3, 2)
    attn_map = 1*(1-attn_map**0.7).reshape(attn_map.shape + (1,))*img + \
            (attn_map**0.7).reshape(attn_map.shape+(1,)) * attn_map_c
    del attn_map_c, cmap, blur
    return attn_map

def viz_attn(img, attn_map, blur=True, title="titre"):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(img)
    axes[1].imshow(getAttMap(img, attn_map, blur))
    for ax in axes:
        ax.axis("off")
    fig.suptitle(title, fontsize=12)
    pdf.savefig(fig)
    plt.close(fig)
    del fig,axes, img, attn_map, blur, title
    
def load_image(img_path, resize=None):
    image = PIL.Image.open(img_path).convert("RGB")
    if resize is not None:
        image = image.resize((resize, resize))
    del resize
    return np.asarray(image).astype(np.float32) / 255.

In [6]:
class Hook:
    """Attaches to a module and records its activations and gradients."""

    def __init__(self, module: nn.Module):
        self.data = None
        self.hook = module.register_forward_hook(self.save_grad)
        
    def save_grad(self, module, input, output):
        self.data = output
        output.requires_grad_(True)
        output.retain_grad()
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.hook.remove()
        
    @property
    def activation(self) -> torch.Tensor:
        return self.data
    
    @property
    def gradient(self) -> torch.Tensor:
        return self.data.grad

In [7]:
class GradCamViT:
    def __init__(self, model, target, height=24):
        self.model = model.eval()
        self.feature = None 
        self.gradient = None r
        self.handlers = [] 
        self.target = target
        self._get_hook() 
        self.height=height

    def _get_features_hook(self, module, input, output):
        if isinstance(output, tuple):
            output=output[0]
        self.feature = self.reshape_transform(output)

    def _get_grads_hook(self, module, input_grad, output_grad):
        if isinstance(output_grad, tuple):
            output_grad=output_grad[0]
        self.gradient = self.reshape_transform(output_grad)

        def _store_grad(grad):
            self.gradient = self.reshape_transform(grad) 


    def _get_hook(self):
        self.target.register_forward_hook(self._get_features_hook)
        self.target.register_forward_hook(self._get_grads_hook)

    def reshape_transform(self, tensor):
        if tensor.shape[1]==1:
            tensor = tensor.permute(1, 0, 2)
        result = tensor[:, 1:, :].reshape(tensor.size(0), self.height, self.height, tensor.size(2))
        result = result.transpose(2, 3).transpose(1, 2) 
        return result

    def __call__(self, inputs, val):
        self.model.zero_grad()  
        if True:
            image=inputs["im"]
            text=inputs["txt"]
            target_features=inputs["txtF"]
            self.model.visual.eval()
            image_features = self.model.visual(image)
            image_features_norm = image_features.norm(dim=-1, keepdim=True)
            image_features_new = image_features / image_features_norm
            target_features_norm = target_features.norm(dim=-1, keepdim=True)
            target_features_new = target_features / target_features_norm
            similarity = image_features_new[0].dot(target_features_new[0])
            self.model.visual.zero_grad()
            similarity.backward(retain_graph=True)
            
            image_attn_blocks = list(dict(self.model.visual.transformer.resblocks.named_children()).values())
            num_tokens = image_attn_blocks[0].attn_probs.shape[-1]
            
            
            cam = image_attn_blocks[-1].attn_grad.detach()
            
            print(cam.shape)
            cam = cam.mean(dim=0) 
            print(cam.shape)
            image_relevance = cam[0, 1:]
            print(image_relevance.shape)
            image_relevance = image_relevance.reshape(1, 1, 24, 24)
            print(image_relevance.shape)
            image_relevance = torch.nn.functional.interpolate(image_relevance, size=224, mode='bilinear')
            print(image_relevance.shape)
            image_relevance = image_relevance.reshape(224, 224)
            print(image_relevance.shape)
            image_relevance = (image_relevance - image_relevance.min()) / (image_relevance.max() - image_relevance.min())
            print(image_relevance.shape)
            cam=image_relevance
            #image=cv2.resize(image, (224,224), interpolation=cv2.INTER_LINEAR)
            #cam = image_relevance * image
            #cam = cam / torch.max(cam)
            #cam = cam[0].permute(1, 2, 0).data.cpu().numpy()
            #cam = np.float32(cam)
            print(cam.max(), cam.min(), cam)
            return cam.cpu().data.numpy()


        #textes=inputs["txt"]
        #produit=[]
        #for t in textes:
        #    inputs["txt"]=t
        #    produit.append(forward(self.model,inputs))
        #print("dfghj")
        #prod=torch.tensor(produit)
        #print("dfghj")
        #target=F.softmax(prod, dim=0)
        #print(target.shape)
        #output = forward(inputs) 

        #index = np.argmax(output.cpu().data.numpy())
        #target = output[0][index]  # Get the target score

        gradient = self.gradient[0].cpu().data.numpy()
        print(gradient.shape)
        weight = np.mean(gradient, axis=(1, 2))  
        print(weight.shape)
        feature = self.feature[0].cpu().data.numpy()
        print(feature.shape)
        
        cam = feature * weight[:, np.newaxis, np.newaxis]
        print(cam.shape)
        cam = np.sum(cam, axis=0) 
        print(cam.shape)
        cam = np.maximum(cam, 0) 
        print(cam.shape)

        cam -= np.min(cam)
        print(cam.shape)
        cam /= np.max(cam)
        print(cam.shape)
        return cam  
        

In [8]:
def gradCAMResnet(
    model: nn.Module,
    input: torch.Tensor,
    target: torch.Tensor,
    layer: nn.Module
) -> torch.Tensor:
    if input.grad is not None:
        input.grad.data.zero_()
        
    requires_grad = {}
    for name, param in model.named_parameters():
        requires_grad[name] = param.requires_grad
        param.requires_grad_(False)
        
    assert isinstance(layer, nn.Module)
    with Hook(layer) as hook:        
        output = model(input)
        output.backward(target)

        grad = hook.gradient.float()
        act = hook.activation.float()
    
        alpha = grad.mean(dim=(2, 3), keepdim=True)
        gradcam = torch.sum(act * alpha, dim=1, keepdim=True)
        gradcam = torch.clamp(gradcam, min=0)

    gradcam = F.interpolate(
        gradcam,
        input.shape[2:],
        mode='bicubic',
        align_corners=False)
    
    for name, param in model.named_parameters():
        param.requires_grad_(requires_grad[name])
        
    return gradcam

In [9]:
def camGradif(cam, im):
    im=ClipImageTrainProcessor(image_size=336)(im)
    
    cam_resized = cv2.resize(cam, (336,336), interpolation=cv2.INTER_LINEAR)  # Resize to match the input image size

    _, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(im.permute(1,2,0).cpu())
    axes[1].imshow(-cam_resized)
    for ax in axes:
        ax.axis("off")
    plt.show()
    
    #im2=np.array(im.permute(1, 2, 0).cpu())
    #plt.imshow(im2,cmap='gray')
    #plt.show()
    
    #combined_image = cv2.addWeighted(im2.astype(np.float32), 0.5, cam_colored.astype(np.float32), 0.5, 0)
    #plt.imshow(combined_image,cmap='gray')
    #plt.show()



def pre_forward_resnet(model,dico, preprocess):
    blur = True #@param {type:"boolean"}
    image_input = preprocess(dico["im"]).unsqueeze(0)
    text_input = clip.tokenize([dico["txt"]])
    text_input=model.encode_text(text_input).float()
    return image_input, text_input

def afficheDiff(cam1,cam2, im):
    cam_différence = (cam1 - cam2)*255

    _, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(im)
    axes[1].imshow(getAttMap(im, cam_différence, True))
    for ax in axes:
        ax.axis("off")
    plt.show()

def lancerCode(vit, chekpoint, im, txt, image_path, title):
    model,preprocess=load_Fmodel_clip(chekpoint=chekpoint, vit=vit)
    #if vit and chekpoint:
    #    text = clip.tokenize(txt)
    #    text_feat,_ = model.encode_text(text)
    #    cam=GradCamViT(model, model.visual.transformer.resblocks[-1])({"im":im,"txt":text, "txtF":text_feat}, chekpoint)
    #    image_np=load_image(image_path, 224)
    #    cam = cv2.resize(cam, (224,224), interpolation=cv2.INTER_LINEAR)
    #    viz_attn(image_np, cam, True, title)
    if vit:
        image = ClipImageTrainProcessor(image_size=336)(im).unsqueeze(0)
        model.add_cls_token=False
        text = clip.tokenize(txt)
        text_feat,_ = model.encode_text(text)
        
        cam=GradCamViT(model, model.visual.transformer.resblocks[-1], height=24)({"im":image,"txt":text, "txtF":text_feat}, chekpoint)
        #inputs={"im":image,"txt":text}
        #inputs = preprocess(text=[txt], images=[im], return_tensors="pt", padding=True)
        #cam=GradCamViT(model, model.visual.transformer.resblocks[-1], height=16)(inputs, chekpoint)
        image_np=load_image(image_path, 224)
        cam = cv2.resize(cam, (224,224), interpolation=cv2.INTER_LINEAR)
        viz_attn(image_np, cam, True, title)
    else:
        image, text=pre_forward_resnet(model,{"im":im,"txt":txt}, preprocess)
        cam=gradCAMResnet(model.visual,image,text,getattr(model.visual, "layer4"))
        cam = cam.squeeze().detach().cpu().numpy()
        image_np=load_image(image_path, model.visual.input_resolution)
        viz_attn(image_np, cam, True, title)
    return cam, image_np

image_path="image/imageFC.png"#"image.jpg"
image=PIL.Image.open(image_path)

im = image#"They are framed by columns, and are looking towards the detailed city and landscape in the background."#,"The Descent from the Cross (or Deposition of Christ, or Descent of Christ from the Cross) is a panel painting by the Flemish artist Rogier van der Weyden created c. 1435, now in the Museo del Prado, Madrid."]

#cam1, image = lancerCode(True, True, im, "Subleyras’s female portraits are recognizable by the round, youthful faces of the models, a characteristic attributed here to Countess Mahony.", image_path)
#cam2, image = lancerCode(True, False, im, "Subleyras’s female portraits are recognizable by the round, youthful faces of the models, a characteristic attributed here to Countess Mahony.", image_path)
#afficheDiff(cam1,cam2, image)

phrase = "A great lady in negligee is intimate and the velvet drapery recall the social status of the character."

# Diviser la phrase en mots
mots = phrase.split()

# Résultats pour chaque mot
vit = True
chekpoint= False
cam_phrase, image_phrase = lancerCode(vit, chekpoint, im, phrase, image_path, phrase)
    cam_phrase, image_phrase = lancerCode(vit, chekpoint, im, phrase, image_path, phrase)
    cam_phrase, image_phrase = lancerCode(vit, chekpoint, im, phrase, image_path, phrase)

torch.Size([16, 577, 577])
torch.Size([577, 577])
torch.Size([576])
torch.Size([1, 1, 24, 24])
torch.Size([1, 1, 224, 224])
torch.Size([224, 224])
torch.Size([224, 224])
tensor(1.) tensor(0.) tensor([[0.2857, 0.2857, 0.2857,  ..., 0.2718, 0.2718, 0.2718],
        [0.2857, 0.2857, 0.2857,  ..., 0.2718, 0.2718, 0.2718],
        [0.2857, 0.2857, 0.2857,  ..., 0.2718, 0.2718, 0.2718],
        ...,
        [0.2859, 0.2859, 0.2859,  ..., 0.3048, 0.3048, 0.3048],
        [0.2859, 0.2859, 0.2859,  ..., 0.3048, 0.3048, 0.3048],
        [0.2859, 0.2859, 0.2859,  ..., 0.3048, 0.3048, 0.3048]])


/tmp/ipykernel_52386/681062032.py:11: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  attn_map = filters.gaussian_filter(attn_map, 0.02*max(img.shape[:2]))


torch.Size([16, 577, 577])
torch.Size([577, 577])
torch.Size([576])
torch.Size([1, 1, 24, 24])
torch.Size([1, 1, 224, 224])
torch.Size([224, 224])
torch.Size([224, 224])
tensor(1.) tensor(0.) tensor([[0.3842, 0.3842, 0.3842,  ..., 0.4225, 0.4225, 0.4225],
        [0.3842, 0.3842, 0.3842,  ..., 0.4225, 0.4225, 0.4225],
        [0.3842, 0.3842, 0.3842,  ..., 0.4225, 0.4225, 0.4225],
        ...,
        [0.4340, 0.4340, 0.4340,  ..., 0.4480, 0.4480, 0.4480],
        [0.4340, 0.4340, 0.4340,  ..., 0.4480, 0.4480, 0.4480],
        [0.4340, 0.4340, 0.4340,  ..., 0.4480, 0.4480, 0.4480]])
torch.Size([16, 577, 577])
torch.Size([577, 577])
torch.Size([576])
torch.Size([1, 1, 24, 24])
torch.Size([1, 1, 224, 224])
torch.Size([224, 224])
torch.Size([224, 224])
tensor(1.) tensor(0.) tensor([[0.6799, 0.6799, 0.6799,  ..., 0.6754, 0.6754, 0.6754],
        [0.6799, 0.6799, 0.6799,  ..., 0.6754, 0.6754, 0.6754],
        [0.6799, 0.6799, 0.6799,  ..., 0.6754, 0.6754, 0.6754],
        ...,
        [0.68